In [2]:
from string import punctuation
from collections import Counter
from tqdm.auto import tqdm
import json
import os

import gensim
import numpy as np
import pandas as pd

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

from nltk.tokenize import MWETokenizer, word_tokenize

import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Описание

Выберите корпус отзывов на товары одной из категорий Amazon:
http://jmcauley.ucsd.edu/data/amazon/


Допустим, что вам нужно подготовить аналитический отчет по этим отзывам — например, для производителя нового продукта этой категории. Для этого будем искать упоминания товаров в отзывах (будем считать их NE). Учтите, что упоминание может выглядеть не только как "Iphone 10", но и как "модель", "телефон" и т.п.

**Важное замечание**: в задании приводятся примеры решений, вы можете их использовать!

In [3]:
def str2dict(s: str):
    return json.loads(s[0:-1])


path_text = "/kaggle/input/amazon-beauty-reviews/All_Beauty.json"
df_text = pd.DataFrame([str2dict(i) for i in open(path_text)])

In [4]:
df_text.tail()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
371340,1.0,True,"07 20, 2017",A202DCI7TV1022,B01HJEGTYK,Sam,It was awful. It was super frizzy and I tried ...,It was super frizzy and I tried to comb it and...,1500508800,NaN,NaN,NaN
371341,5.0,True,"03 16, 2017",A3FSOR5IJOFIBE,B01HJEGTYK,TYW,I was skeptical about buying this. Worried it...,Awesome,1489622400,34,NaN,NaN
371342,5.0,True,"03 1, 2017",A1B5DK6CTP2P24,B01HJEGTYK,Norma Jennings,Makes me look good fast.,Five Stars,1488326400,46,NaN,NaN
371343,2.0,True,"02 21, 2017",A23OUYS5IRMJS9,B01HJEGTYK,Lee,Way lighter than photo\nNot mix blend of color...,Ok but color way off and volume as well,1487635200,NaN,NaN,NaN
371344,2.0,True,"12 15, 2016",A24KQ9RVU81L87,B01HJEGTYK,Season341,No return instructions/phone # in packaging. ...,Might return for a replacement if I could.,1481760000,NaN,NaN,NaN


In [5]:
df_text['review'] = df_text['reviewText'] + ' ' + df_text['summary']
df_text = df_text[['review']]

In [6]:
df_text.shape

(371345, 1)

In [7]:
path_meta = "/kaggle/input/amazon-fashion-meta/meta_All_Beauty.json"
df_meta = pd.DataFrame([str2dict(i) for i in open(path_meta)])
df_meta.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [8]:
metadata = df_meta[df_meta['title'] != '']['title'].tolist()

In [9]:
metadata[:5]

["Loud 'N Clear&trade; Personal Sound Amplifier",
 'No7 Lift &amp; Luminate Triple Action Serum 50ml by Boots',
 'No7 Stay Perfect Foundation Cool Vanilla by No7',
 'Wella Koleston Perfect Hair Colour 44/44 Medium Intense Red Brown 60ml',
 'Lacto Calamine Skin Balance Oil control 120 ml. (Pack of 2)']

## 1. ✅ (3 балла) Предложите 3 способа найти упоминания товаров в отзывах. 
Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара.
Могут помочь заголовки и дополнительные данные с Amazon (Metadata [здесь](https://nijianmo.github.io/amazon/index.html)). Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?

    1.  Dictionary-based подход
    
    Достоинства: простота (типично для бейслайна), хорошо работает со статичными типами entities, мало неоднозначности, скорость

    Недостатки: нужны тексты стандартного вида, область должна быть предметно ограничена, плохо работает с динамичными классами entities, словарь надо делать чем больше, тем лучше (возможно придется еще делать fuzzy search)
    
    2. Rule-based подход (например, Yargy)
    
    Достоинства: легко формализовать сущности, мало неоднозначности, скорость, в правилаъ можно комбинировать точные вхождения и pos
    
    Недостатки: времязатратность, нужны тексты стандартного вида, область должна быть предметно ограничена
    
    3. Classification-based
    
    Достоинства: определяет то, что не было на трейне; отлично работает с single-token entities, 
    
    Недостатки: проблемы с multi-token entities, требует большое количество размеченный training data (нужно создавать и аннотировать корпус),  
    
    2. SpaCy
    
    Достоинства: мультиязычность, сокрость
    
    Недостатки: нет

## 2. (2 балла) Реализуйте один из предложенных вами способов.

Примеры в качестве подсказки (можно использовать один из них): 
- написать правила с помощью [natasha/yargy](https://github.com/natasha/yargy)
- составить мини-словарь сущностей/дескрипторов, расширить с помощью эмбеддингов (например, word2vec)

In [10]:
def preprocessing(text):
    text = text.lower()
    for punct in punctuation:
        text = text.replace(punct, ' ')
    text = text.split()
    return text

In [11]:
corpora = [preprocessing(t) for t in df_text['review'].tolist() if type(t) != float]
corpora_str = [' '.join(text) for text in corpora]
print(corpora[0], corpora_str[0], sep='\n')

['great', 'one', 'star']
great one star


In [12]:
bow = []
for words in tqdm(corpora):
    for word in words:
        if word not in punctuation and word not in stopwords_en:
            bow.append(word)

  0%|          | 0/370756 [00:00<?, ?it/s]

In [13]:
wordcount = Counter(bow)

In [14]:
wordcount.most_common(20)

[('product', 126014),
 ('great', 122261),
 ('stars', 101888),
 ('love', 87443),
 ('like', 81946),
 ('good', 81623),
 ('use', 78998),
 ('one', 72954),
 ('five', 71938),
 ('hair', 67810),
 ('skin', 59305),
 ('well', 49520),
 ('really', 48990),
 ('works', 48063),
 ('would', 46526),
 ('get', 43255),
 ('used', 42544),
 ('using', 39645),
 ('time', 38453),
 ('nice', 37011)]

Как и предполагалось, слово _product_ действительно супер частотное. В принципе выделяются нужные нам слова (_hair, skin_), но есть много мусора. 

**Сделаем bow из названий из брендов.**

In [27]:
import re

bow_meta = []
for words in tqdm(metadata):
    for word in words.split():
        word = word.lower()
        if (
            word not in stopwords_en and 
            word not in punctuation and 
            len(word) > 2 and
            len(re.findall(r'[^\w\s]', word)) == 0
        ):
            bow_meta.append(word)

  0%|          | 0/32891 [00:00<?, ?it/s]

In [28]:
wordcount_meta = Counter(bow_meta)

In [56]:
wordcount_meta.most_common(20)

[('hair', 3796),
 ('nail', 2238),
 ('body', 2098),
 ('set', 2011),
 ('black', 1690),
 ('oil', 1458),
 ('skin', 1455),
 ('cream', 1400),
 ('makeup', 1348),
 ('color', 1338),
 ('natural', 1271),
 ('pack', 1271),
 ('gel', 1243),
 ('women', 1180),
 ('brush', 1164),
 ('lip', 1141),
 ('spray', 1002),
 ('ounce', 996),
 ('face', 943),
 ('eye', 937)]

Получился добротный список: много упоминаний нужных продуктов. Хотя мы видим много прилагательных.

In [54]:
key_words = []

for word, freq in  wordcount_meta.most_common(100):
    if nlp(word)[0].pos_ == 'NOUN':
        key_words.append(word)

In [55]:
key_words

['hair',
 'body',
 'oil',
 'skin',
 'cream',
 'makeup',
 'color',
 'women',
 'ounce',
 'face',
 'eye',
 'kit',
 'art',
 'beauty',
 'soap',
 'size',
 'case',
 'fashion',
 'lotion',
 'care',
 'gift',
 'bath',
 'travel',
 'silver',
 'hand',
 'mask',
 'gold',
 'shampoo',
 'men',
 'colors',
 'liquid',
 'treatment',
 'style',
 'bar',
 'steel',
 'collection',
 'ring',
 'water',
 'flower',
 'tool',
 'shower',
 'mini',
 'jewelry',
 'perfume',
 'manicure',
 'rhinestone',
 'party',
 'holder',
 'fragrance',
 'cosmetics']

**Выделим названия брендов**

In [100]:
brand_names = df_meta['title'].dropna().tolist()
brand_names[:10]

["Loud 'N Clear&trade; Personal Sound Amplifier",
 'No7 Lift &amp; Luminate Triple Action Serum 50ml by Boots',
 'No7 Stay Perfect Foundation Cool Vanilla by No7',
 'Wella Koleston Perfect Hair Colour 44/44 Medium Intense Red Brown 60ml',
 'Lacto Calamine Skin Balance Oil control 120 ml. (Pack of 2)',
 'Mary Kay Satin Hands Hand Cream Travel MINI Size Set of 6',
 'Unique Custom Cast Iron Liner Shader Tattoo Machine E010541',
 'Imagen Bendita Por Su Santidad Our Lady of Charity Caridad Del Cobre Patrona De Cuba Cachita B-36 Small - 6 Inches',
 'Mali princ',
 "Moleskine Payne's Grey Card Wallet (Moleskine Non-Paper)"]

Сложность в том, что названия достаточно длинные (в них есть указания объема и вся-вся другая информация). Но как мы видим, обычно название бренда обычно содержится в первом и втором слове (No7, Lacto, Moleskin и т.п.) Так что будем условно считатать, что название бренда == 2 первых слова.

In [139]:
brand_names_filtered = []
trash = stopwords_en + list(punctuation)

for brand in tqdm(brand_names):
    brand_split = brand.split()
    if (len(brand_split) >= 2 and 
        re.match(r'[^0-9]', brand_split[0]) and
        re.match(r'[^0-9]', brand_split[1]) and
        brand_split[0] not in trash and
        brand_split[1] not in trash
       ):
        brand_names_filtered.append(' '.join(brand_split[:2]).lower()) 

  0%|          | 0/32892 [00:00<?, ?it/s]

In [140]:
brand_names_set = Counter(brand_names_filtered).most_common(200)

In [141]:
brand_names_set

[('(3 pack)', 195),
 ('(6 pack)', 110),
 ('sally hansen', 97),
 ('bath &amp;', 93),
 ('philips norelco', 92),
 ('betty dain', 85),
 ('wet n', 79),
 ('born pretty', 72),
 ('estee lauder', 66),
 ("victoria's secret", 59),
 ('mary kay', 57),
 ('old spice', 48),
 ('bare escentuals', 46),
 ('nailart nail', 43),
 ('so beauty', 42),
 ('l.a. colors', 39),
 ('sephora collection', 38),
 ('ultimate arms', 37),
 ('greenwich bay', 36),
 ('iluve long', 33),
 ('elite99 soak', 32),
 ('dnd gel', 32),
 ('pre de', 30),
 ('kevin murphy', 30),
 ('water nail', 30),
 ('city color', 30),
 ('hard candy', 29),
 ('women perfume', 28),
 ('earthly body', 27),
 ('bmc nail', 27),
 ('topline autopart', 27),
 ('max factor', 26),
 ('lcl beauty', 26),
 ('boots no7', 25),
 ('china glaze', 25),
 ('physicians formula', 25),
 ('hello kitty', 24),
 ('bobbi brown', 24),
 ('gold bond', 24),
 ("l'oreal paris", 23),
 ("claire's accessories", 23),
 ('the body', 22),
 ('sleek makeup', 21),
 ('charlotte tilbury', 21),
 ('piping roc

In [142]:
brand_in_text = []
for text in tqdm(corpora_str):
    for brand, freq in brand_names_set:
        if brand in text:
            brand_in_text.append(brand)

  0%|          | 0/370756 [00:00<?, ?it/s]

In [152]:
key_words_brands = list(set(brand_in_text))

In [154]:
key_words_brands

['nail art',
 'kevin murphy',
 'brand new',
 'peter thomas',
 'coochy rash',
 'ag hair',
 'star wars',
 'sephora brand',
 'black opal',
 'estee lauder',
 'so beauty',
 'ys park',
 'christian dior',
 'hard candy',
 'hawaiian tropic',
 'cover girl',
 'benefit cosmetics',
 'signature club',
 'harvey prince',
 'earth therapeutics',
 'dead sea',
 'max factor',
 'kama sutra',
 'tea tree',
 'great value',
 'earthly body',
 'fa deodorant',
 'avon skin',
 'france luxe',
 'john frieda',
 'banana boat',
 'barry m',
 'schwarzkopf igora',
 'mary kay',
 'citre shine',
 'the body',
 'too faced',
 'dnd gel',
 'kiara sky',
 'world traveler',
 'victoria secret',
 'gillette venus',
 'garnier fructis',
 'lip smacker',
 'tigi bed',
 'william marvy',
 'bobbi brown',
 'physicians formula',
 'now foods',
 'camille beckman',
 'doinshop new',
 'born pretty',
 'andalou naturals',
 'gosh velvet',
 'boots botanics',
 'au naturale',
 'alba botanica',
 'sleek make',
 'nyc expert',
 'queen helene',
 'hair straightene

Мы получили названия брендов! Добавим их к списку ключевых слов.

In [155]:
KEYWORDS = key_words + key_words_brands

In [156]:
len(KEYWORDS)

206

In [159]:
KEYWORDS[-20:]

['eg gifts',
 'pure romance',
 'lip ink',
 'tom ford',
 'designer skin',
 'greenwich bay',
 'laura mercier',
 'sleek makeup',
 'femme couture',
 'maybelline new',
 'bare minerals',
 'sexy hair',
 'argan oil',
 'essential oil',
 'pink pewter',
 'urban decay',
 'american crew',
 'clairol nice',
 'h2o plus',
 'bare escentuals']

**Обучаем w2v**

Можем еще обучить w2v и найти слвоа, похожие на самое популярное слово _product_.

In [160]:
model = gensim.models.Word2Vec(corpora, window=3, sg=0)

In [161]:
model.wv.most_similar('product', topn=15)

[('item', 0.8099555373191833),
 ('stuff', 0.7295476198196411),
 ('device', 0.6904293298721313),
 ('serum', 0.6481928825378418),
 ('deodorant', 0.6069017052650452),
 ('company', 0.575533926486969),
 ('unit', 0.5667423009872437),
 ('toner', 0.5654284358024597),
 ('shaver', 0.5531589388847351),
 ('cream', 0.5514410138130188),
 ('wig', 0.5495710372924805),
 ('seller', 0.5443364977836609),
 ('model', 0.5436307191848755),
 ('moisturizer', 0.5401099324226379),
 ('machine', 0.5396619439125061)]

**Посмотрим на самые частотные слова: может, среди них есть то, что нам нужно для выделения NE**

In [162]:
print('product:', wordcount['product'])
for i in model.wv.most_similar('product', topn=10):
    print(i[0], ': ', wordcount[i[0]], sep='')

product: 126014
item: 10416
stuff: 16104
device: 2097
serum: 3761
deodorant: 6164
company: 4418
unit: 5026
toner: 1600
shaver: 13162
cream: 14788


В п. 5 нам нужно будет анализировать 5 товаров, так что помимо _product_ запоминаем _item, stuff, cream, unit, moisturizer_.

In [163]:
top_words = ['product', 'item', 'stuff', 'cream', 'unit', 'moisturizer']

## 3. ✅ (1 балл) Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

In [183]:
tokenizer = MWETokenizer([tuple(w.split()) for w in KEYWORDS if len(w.split()) > 1])

In [219]:
bigrams = []
# lemmas = []

for text in tqdm(corpora):
    tokens = tokenizer.tokenize(text)
    # lemmas.append(tokens)
    
    text_bigrams = list(nltk.bigrams(tokens))
    for bigram in text_bigrams:
        if bigram[0] in punctuation or bigram[1] in punctuation:
            continue
        elif bigram[0] in KEYWORDS or bigram[1] in KEYWORDS:

  0%|          | 0/370756 [00:00<?, ?it/s]

In [221]:
bigrams[-10:]

[('my', 'hair'),
 ('hair', 'looks'),
 ('of', 'colors'),
 ('colors', 'not'),
 ('again', 'color'),
 ('color', 'is'),
 ('but', 'color'),
 ('color', 'way'),
 ('packaging', 'color'),
 ('color', 'i')]

In [223]:
Counter(bigrams).most_common(10)

[(('my', 'skin'), 21158),
 (('my', 'hair'), 21007),
 (('my', 'face'), 12251),
 (('the', 'color'), 7113),
 (('skin', 'and'), 5639),
 (('hair', 'and'), 5636),
 (('the', 'skin'), 5308),
 (('your', 'skin'), 5283),
 (('sensitive', 'skin'), 4756),
 (('the', 'hair'), 4704)]

## 4. (3 балла) Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). 
Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика  ранжирует выше коллокации, подходящие для отчёта).

Я считаю likelihood ratio, PMI и student_t.

In [236]:
nltk.download('omw-1.4')

from nltk.collocations import *
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [244]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = BigramCollocationFinder.from_documents(corpora)
bigram_finder.apply_freq_filter(10)

In [265]:
rate_pmi = []
for word in tqdm(top_words):
    lem = lemmatizer.lemmatize(word)
    rate = [i for i in bigram_finder.nbest(bigram_measures.pmi, 10**10) if lem in i][:100]
    rate_pmi.append(rate)

  0%|          | 0/6 [00:00<?, ?it/s]

In [250]:
rate_likelihood = []
for word in tqdm(top_words):
    lem = lemmatizer.lemmatize(word)
    rate = [i for i in bigram_finder.nbest(bigram_measures.likelihood_ratio, 10**10) if lem in i][:100]
    rate_likelihood.append(rate)

  0%|          | 0/6 [00:00<?, ?it/s]

In [251]:
rate_t = []
for word in tqdm(top_words):
    lem = lemmatizer.lemmatize(word)
    rate = [i for i in bigram_finder.nbest(bigram_measures.student_t, 10**10) if lem in i][:100]
    rate_t.append(rate)

  0%|          | 0/6 [00:00<?, ?it/s]

## 5. ✅ (1 балл) Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров.
Должны получиться примерно такие группы:

```
watch 
--- 
stylish watch
good watches
great watch
love this watch
...
```

In [ ]:
columns = []
for measure in ['pmi', 't', 'lh']:
    columns.extend([f'{word}_{measure}' for word in top_words])
    
df_rate = pd.DataFrame(columns=columns)
for i in range(6):
    df_rate[f'{top_words[i]}_pmi'] = rate_pmi[i]
    df_rate[f'{top_words[i]}_t'] = rate_t[i]
    df_rate[f'{top_words[i]}_lh'] = rate_likelihood[i]

In [280]:
df_rate

,product_pmi,item_pmi,stuff_pmi,cream_pmi,unit_pmi,moisturizer_pmi,product_t,item_t,stuff_t,cream_t,unit_t,moisturizer_t,product_lh,item_lh,stuff_lh,cream_lh,unit_lh,moisturizer_lh
0,"(hook, product)","(novelty, item)","(diva, stuff)","(bb, cream)","(tens, unit)","(tinted, moisturizer)","(this, product)","(this, item)","(this, stuff)","(this, cream)","(the, unit)","(a, moisturizer)","(this, product)","(this, item)","(this, stuff)","(eye, cream)","(the, unit)","(facial, moisturizer)"
1,"(gp, product)","(luxury, item)","(stuff, stinks)","(cc, cream)","(steam, unit)","(daytime, moisturizer)","(great, product)","(the, item)","(stuff, is)","(eye, cream)","(this, unit)","(this, moisturizer)","(great, product)","(great, item)","(stuff, is)","(this, cream)","(this, unit)","(tinted, moisturizer)"
2,"(product, link)","(wrong, item)","(stuff, rocks)","(progesterone, cream)","(defective, unit)","(nighttime, moisturizer)","(the, product)","(great, item)","(good, stuff)","(shaving, cream)","(unit, is)","(great, moisturizer)","(good, product)","(the, item)","(good, stuff)","(shaving, cream)","(unit, is)","(a, moisturizer)"
3,"(product, junkie)","(item, arrived)","(nasty, stuff)","(kukui, cream)","(portable, unit)","(retinol, moisturizer)","(good, product)","(item, is)","(great, stuff)","(cream, is)","(new, unit)","(moisturizer, and)","(the, product)","(item, arrived)","(great, stuff)","(bb, cream)","(new, unit)","(daily, moisturizer)"
4,"(excellent, product)","(correct, item)","(this, stuff)","(glossing, cream)","(main, unit)","(facial, moisturizer)","(product, is)","(item, was)","(stuff, works)","(the, cream)","(old, unit)","(moisturizer, i)","(excellent, product)","(item, was)","(stuff, works)","(night, cream)","(old, unit)","(great, moisturizer)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"(product, performed)","(item, its)","(stuff, in)","(based, cream)","(unit, to)","(moisturizer, from)","(organic, product)","(item, would)","(black, stuff)","(lip, cream)","(unit, now)","(different, moisturizer)","(product, using)","(expensive, item)","(stuff, dries)","(gel, cream)","(unit, since)","(moisturizer, by)"
96,"(product, shipped)","(item, but)","(stuff, absolutely)","(cream, which)","(unit, or)","(moisturizer, doesn)","(product, based)","(item, looked)","(getting, stuff)","(cuticle, cream)","(great, unit)","(your, moisturizer)","(product, smells)","(item, please)","(stuff, feels)","(skin, cream)","(unit, still)","(moisturizer, makes)"
97,"(misleading, product)","(1, item)","(stuff, however)","(lip, cream)","(unit, than)","(or, moisturizer)","(authentic, product)","(little, item)","(stuff, gives)","(butt, cream)","(unit, or)","(moisturizer, then)","(fabulous, product)","(find, item)","(terrible, stuff)","(shower, cream)","(unit, back)","(moisturizer, four)"
98,"(beauty, product)","(item, 4)","(new, stuff)","(cream, base)","(unit, just)","(your, moisturizer)","(free, product)","(favorite, item)","(new, stuff)","(repair, cream)","(unit, just)","(over, moisturizer)","(product, little)","(cute, item)","(stuff, made)","(rich, cream)","(unit, even)","(different, moisturizer)"


In [267]:
df_rate

,product_pmi,item_pmi,stuff_pmi,cream_pmi,unit_pmi,moisturizer_pmi,product_t,item_t,stuff_t,cream_t,unit_t,moisturizer_t,product_lh,item_lh,stuff_lh,cream_lh,unit_lh,moisturizer_lh
0,"(hook, product)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"(gp, product)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"(product, link)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"(product, junkie)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"(excellent, product)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"(product, performed)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,"(product, shipped)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,"(misleading, product)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,"(beauty, product)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 6. (2 балла) Бонус 
если придумаете способ объединить синонимичные упоминания (например, "Samsung Galaxy Watch", "watch", "smartwatch")